In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("tourism.csv").drop(columns=["Unnamed: 0"])

#Code taken from:
#https://towardsdatascience.com/introduction-to-hierarchical-time-series-forecasting-part-ii-e73dfd3de86b

In [6]:
# lowercase the column names
df.columns = [col_name.lower() for col_name in df.columns]

# sum the trips over purpose
df = df.groupby(["quarter", "region", "state"])["trips"] \
       .sum() \
       .reset_index(drop=False)

# cleanup region name 
df["region"] = df["region"].apply(lambda x: x.replace(" ", "_").replace("'", ""))

# map the full state names to abbreviations
mapping_dict = {
    "New South Wales": "NSW",
    "Northern Territory": "NT",
    "Queensland": "QLD",
    "South Australia": "SA",
    "Victoria": "VIC",
    "Western Australia": "WA",
    "ACT": "ACT",
}

df["state"] = df["state"].map(mapping_dict)

# create the bottom level id
df["state_region"] = df.apply(lambda x: f"{x['state']}_{x['region']}", axis=1)

In [7]:

# create the bottom level df
df_bottom_level = df.pivot(index="quarter", columns="state_region", values="trips")

# create the middle level df
df_middle_level = df.groupby(["quarter", "state"]) \
                    .sum() \
                    .reset_index(drop=False) \
                    .pivot(index="quarter", columns="state", values="trips")

# create the total level df
df_total = df.groupby("quarter")["trips"] \
             .sum() \
             .to_frame() \
             .rename(columns={"trips": "total"})

# join the DataFrames
hierarchy_df = df_bottom_level.join(df_middle_level) \
                              .join(df_total)
hierarchy_df.index = pd.to_datetime(hierarchy_df.index)
hierarchy_df = hierarchy_df.resample("QS") \
                           .sum()

print(f"Number of time series at the bottom level: {df_bottom_level.shape[1]}")
print(f"Number of time series at the middle level: {df_middle_level.shape[1]}")

Number of time series at the bottom level: 77
Number of time series at the middle level: 7


In [8]:
hierarchy_df

,ACT_Canberra,NSW_Blue_Mountains,NSW_Capital_Country,NSW_Central_Coast,NSW_Central_NSW,NSW_Hunter,NSW_New_England_North_West,NSW_North_Coast_NSW,NSW_Outback_NSW,NSW_Riverina,...,WA_Australias_South_West,WA_Experience_Perth,ACT,NSW,NT,QLD,SA,VIC,WA,total
quarter,,,,,,,,,,,,,,,,,,,,,
1998-01-01,551.001921,195.543180,261.808142,455.166849,424.395446,674.978457,329.838506,1342.301230,96.602651,215.268912,...,474.858729,751.211958,551.001921,8039.794795,1914.709878,4042.779858,1732.743704,6010.424491,1641.089495,23932.544141
1998-04-01,416.025623,200.408671,238.600029,343.476758,557.011571,791.887234,328.366005,1161.108602,120.648446,336.094921,...,411.622281,668.710707,416.025623,7166.013805,1541.405790,3967.069714,1394.638319,4795.246755,1576.326534,20856.726540
1998-07-01,436.029011,253.359689,184.423752,303.670875,555.527970,566.040599,324.859168,839.368066,110.073779,228.065371,...,360.039657,662.850731,436.029011,6747.935790,1321.819326,4598.548477,1213.330723,4316.845170,1588.293692,20222.802189
1998-10-01,449.798445,245.573846,229.088938,331.817060,590.158435,591.568947,279.796195,1183.859971,185.470023,201.123526,...,462.620050,832.706514,449.798445,7282.082371,1439.406267,4207.772812,1449.354514,4674.829118,1839.716990,21342.960517
1999-01-01,378.572817,290.484124,252.281951,466.815571,480.129260,785.102775,327.491877,1306.877508,78.363807,200.822443,...,562.974629,830.305740,378.572817,7584.776839,1830.389536,4332.490850,1535.459593,5304.334195,1835.687573,22801.711403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-10-01,542.769655,203.906222,369.513080,364.288154,512.524298,821.778303,333.112869,1213.745435,95.876016,208.726401,...,647.774842,937.578580,542.769655,7865.399385,2087.124892,5368.858427,1695.622429,5817.562633,2382.091755,25759.429177
2016-01-01,626.640631,239.080990,303.226820,412.911330,503.693601,858.614110,315.795461,1624.141691,60.299009,243.643135,...,967.389399,1086.068326,626.640631,8407.366451,2114.266717,5095.546237,1874.771647,6576.363923,2797.404767,27492.360374
2016-04-01,592.608499,303.558354,322.504718,250.955972,461.043217,829.977102,392.017225,1153.697747,134.831251,279.953218,...,650.119716,960.572196,592.608499,7660.579621,1628.593948,5444.312492,1661.079180,5335.229737,2469.394281,24791.797758


In [9]:
states = df["state"].unique()
regions = df["state_region"].unique()

total = {'total': list(states)}
state = {k: [v for v in regions if v.startswith(k)] for k in states}
hierarchy = {**total, **state}

In [11]:
hierarchy

{'total': ['SA', 'NT', 'WA', 'VIC', 'NSW', 'QLD', 'ACT'],
 'SA': ['SA_Adelaide',
  'SA_Adelaide_Hills',
  'SA_Barossa',
  'SA_Clare_Valley',
  'SA_Eyre_Peninsula',
  'SA_Fleurieu_Peninsula',
  'SA_Flinders_Ranges_and_Outback',
  'SA_Kangaroo_Island',
  'SA_Limestone_Coast',
  'SA_Murraylands',
  'SA_Riverland',
  'SA_Yorke_Peninsula'],
 'NT': ['NT_Alice_Springs',
  'NT_Barkly',
  'NT_Darwin',
  'NT_East_Coast',
  'NT_Hobart_and_the_South',
  'NT_Kakadu_Arnhem',
  'NT_Katherine_Daly',
  'NT_Lasseter',
  'NT_Launceston,_Tamar_and_the_North',
  'NT_MacDonnell',
  'NT_North_West',
  'NT_Tasmania',
  'NT_Wilderness_West'],
 'WA': ['WA_Australias_Coral_Coast',
  'WA_Australias_Golden_Outback',
  'WA_Australias_North_West',
  'WA_Australias_South_West',
  'WA_Experience_Perth'],
 'VIC': ['VIC_Ballarat',
  'VIC_Bendigo_Loddon',
  'VIC_Central_Highlands',
  'VIC_Central_Murray',
  'VIC_Geelong',
  'VIC_Gippsland',
  'VIC_Goulburn',
  'VIC_High_Country',
  'VIC_Lakes',
  'VIC_Macedon',
  'VIC_Ma

In [14]:
df

,quarter,region,state,trips,state_region
0,1998-01-01,Adelaide,SA,658.553895,SA_Adelaide
1,1998-01-01,Adelaide_Hills,SA,9.798630,SA_Adelaide_Hills
2,1998-01-01,Alice_Springs,NT,20.207638,NT_Alice_Springs
3,1998-01-01,Australias_Coral_Coast,WA,132.516409,WA_Australias_Coral_Coast
4,1998-01-01,Australias_Golden_Outback,WA,161.726948,WA_Australias_Golden_Outback
...,...,...,...,...,...
5847,2016-10-01,Western_Grampians,VIC,102.374415,VIC_Western_Grampians
5848,2016-10-01,Whitsundays,QLD,88.041287,QLD_Whitsundays
5849,2016-10-01,Wilderness_West,NT,30.865882,NT_Wilderness_West
5850,2016-10-01,Wimmera,VIC,25.709937,VIC_Wimmera


In [18]:
df_middle_level = df.groupby(["quarter", "state"]) \
                    .sum() \
                    .reset_index(drop=False)
df_top_level = df.groupby(["quarter"]) \
                    .sum() \
                    .reset_index(drop=False)

In [25]:
df_middle_level

,quarter,state,trips
0,1998-01-01,ACT,551.001921
1,1998-01-01,NSW,8039.794795
2,1998-01-01,NT,1914.709878
3,1998-01-01,QLD,4042.779858
4,1998-01-01,SA,1732.743704
...,...,...,...
527,2016-10-01,NT,1846.582167
528,2016-10-01,QLD,6105.040807
529,2016-10-01,SA,1635.467714
530,2016-10-01,VIC,6110.940085


In [30]:
pd.concat([df, df_middle_level, df_top_level]).fillna("total").drop(columns=["state_region"]).to_csv("cleaned_tourism.csv", index=False)